In [1]:
from llm import OpenAICompatibleChatCompletion, AnthropicLLM
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from ecommerce import questions, ecommerce_metrics, sales_channels, brands
import numpy as np

In [2]:
llm_anthropic = AnthropicLLM()
llm_anthropic.call(
    messages=[{"role": "system", "content": "Talk like a pirate and return JSON"}, {"role": "user", "content": "hey"}], max_tokens=1000
)

{'message': {'content': [{'text': 'Ahoy there, matey! What can I do for ye on this fine day at sea?',
    'type': 'text'}],
  'role': 'assistant'},
 'model': 'claude-3-5-sonnet-20240620',
 'token_usage': {'completion_tokens': 23,
  'prompt_tokens': 16,
  'total_tokens': 39}}

In [3]:
llm_openai = OpenAICompatibleChatCompletion()
llm_openai.call(messages=[{"role": "system", "content": "Talk like a pirate and return JSON"}, {"role": "user", "content": "hey"}])

{'message': {'content': 'Ahoy matey! What can this old sea dog do for ye today? Aarrr! Here be the information ye seek in JSON format:\n\n{\n  "greeting": "Ahoy matey!",\n  "message": "What can this old sea dog do for ye today? Aarrr!"\n}',
  'role': 'assistant'},
 'model': 'gpt-3.5-turbo-0125',
 'token_usage': {'completion_tokens': 63,
  'prompt_tokens': 19,
  'total_tokens': 82}}

In [4]:
def prepare_embedding_input(rec):
    return f'{rec["name"]} {rec["enum"]}'

In [5]:
ecommerce_metrics_embeddings = llm_openai.get_embeddings([prepare_embedding_input(rec) for rec in ecommerce_metrics])
brands_embeddings = llm_openai.get_embeddings([prepare_embedding_input(rec) for rec in brands])

In [6]:
from copy import deepcopy


def convert_openai_tool_to_anthropic(t: dict):
    t = deepcopy(t)
    t = t["function"]
    t["input_schema"] = t["parameters"]
    t.pop("parameters")
    return t

In [7]:
get_backend_metric_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_metric",
        "description": """Takes in the user requested metric and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM metrics.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_metric": {
                    "type": "string",
                    "description": "The metric requested by the user.",
                },
            },
            "required": ["user_requested_metric"],
        },
    },
}
get_backend_brands_tool = {
    "type": "function",
    "function": {
        "name": "get_backend_brands",
        "description": """Takes in the user requested brand(s) and 
        uses ML/AI to return the k nearest neighbors for the most likely related backend ENUM brands per requested brands.""",
        "parameters": {
            "type": "object",
            "properties": {
                "user_requested_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of brand(s) requested by the user.",
                },
            },
            "required": ["user_requested_brands"],
        },
    },
}
get_sales_data_tool = {
    "type": "function",
    "function": {
        "name": "get_sales_data",
        "description": """Get the sales data from the backend system.""",
        "parameters": {
            "type": "object",
            "properties": {
                "backend_metric": {
                    "type": "string",
                    "description": "This is the backend metric ENUM.",
                },
                "backend_brands": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "default": [],
                    "description": "The list of backend ENUM brands.",
                },
                "sales_channels": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": [x["enum"] for x in sales_channels],
                    },
                    "default": [],
                    "description": "The list of sales channels.",
                },
                "current_period_start_date": {
                    "type": "string",
                    "description": "The start of the current reporting period.",
                },
                "current_period_end_date": {
                    "type": "string",
                    "description": "The end of the current reporting period.",
                },
            },
            "required": [
                "backend_metric",
                "backend_brands",
                "current_period_start_date",
                "current_period_end_date",
            ],
        },
    },
}
tools_openai = [get_backend_metric_tool, get_backend_brands_tool, get_sales_data_tool]
tools_anthropic = [convert_openai_tool_to_anthropic(t) for t in tools_openai]

In [8]:
def find_k_nearest_neighbors(embeddings, input_embedding, k):
    # Calculate distances
    distances = np.linalg.norm(embeddings - input_embedding, axis=1)
    # Get indices of k smallest distances
    nearest_indices = np.argpartition(distances, k)[:k]
    # Sort the k nearest indices by distance
    nearest_indices = nearest_indices[np.argsort(distances[nearest_indices])]
    return nearest_indices

In [9]:
def get_backend_metric(user_requested_metric: str):
    return {
        "data": [
            ecommerce_metrics[i]
            for i in find_k_nearest_neighbors(llm_openai.get_embeddings([user_requested_metric]), ecommerce_metrics_embeddings, 3)
        ]
    }


def get_backend_brands(user_requested_brands: list[str]):
    data = dict()
    for brand in user_requested_brands:
        data[brand] = [brands[i] for i in find_k_nearest_neighbors(llm_openai.get_embeddings([brand]), brands_embeddings, 3)]
    return {"data": data}


def get_sales_data(*args, **kwargs):
    return {"data": 10}


functions_look_up = {"get_backend_metric": get_backend_metric, "get_backend_brands": get_backend_brands, "get_sales_data": get_sales_data}

In [10]:
get_backend_brands(["shopify", "nike"])

{'data': {'shopify': [{'name': 'Shopify',
    'enum': 'SHOPIFY',
    'description': 'E-commerce platform for online stores'},
   {'name': 'Etsy',
    'enum': 'ETSY',
    'description': 'E-commerce website focused on handmade or vintage items'},
   {'name': 'Amazon',
    'enum': 'AMAZON',
    'description': 'E-commerce and cloud computing giant'}],
  'nike': [{'name': 'Nike',
    'enum': 'NIKE',
    'description': 'Global sportswear and athletic footwear brand'},
   {'name': 'Nikon',
    'enum': 'NIKON',
    'description': 'Japanese multinational optics and imaging products corporation'},
   {'name': 'Adidas',
    'enum': 'ADIDAS',
    'description': 'German sportswear manufacturer'}]}}

In [11]:
system_prompt = """
You will be asked a question by the user about retrieving sales data.
Use the available tools but only call the tools when needed.
If you need further clarification then ask. 

There are hundreds of metrics and hundreds of brands in the backend system.
The user will not know all these metrics and brands, or how to refer to them exactly.
You do not know them either, so I have provided some helper tools for you.

In general you will follow the typical flow when answering questions:
1. Extract the user requested metric and the user requested brand(s).
2. 
    
    a) Pass the user requested metric to the the tool get_backend_metric to
    get the list of most likely corresponding backend metric ENUMs. 
    Then choose the most appropriate from this list. 
    
    b) Pass the user requested brand(s) to the the tool get_backend_brands to
    get the list of most likely corresponding backend brand ENUMs. 
    Then choose the most appropriate from this list. 
    
3. Extract any sales channels if mentioned.
4. Pass the relevant arguments into the get_sales_data tool.


Today's date is Monday, June 10, 2024
"""

In [12]:
def eval_llm_resp(question: dict, llm_resp: dict):
    if not llm_resp.get("tool_calls_details"):
        args_predicted = dict()
    else:
        args_predicted = [x["input"] for x in llm_resp["tool_calls_details"].values() if x["name"] == "get_sales_data"]
        args_predicted = args_predicted[0] if args_predicted else {}
    return {
        "question": question["question"],
        "expected_metric": question["expected_metric"],
        "predicted_metric": args_predicted.get("backend_metric", ""),
        "metric_correct": question["expected_metric"] == args_predicted.get("backend_metric", ""),
        "expected_brands": sorted(question["expected_brands"]),
        "predicted_brands": sorted(args_predicted.get("backend_brands", [])),
        "brands_correct": sorted(question["expected_brands"]) == sorted(args_predicted.get("backend_brands", [])),
        "expected_sales_channels": sorted(question["expected_sales_channels"]),
        "predicted_sales_channels": sorted(args_predicted.get("sales_channels", [])),
        "sales_channels_correct": sorted(question["expected_sales_channels"]) == sorted(args_predicted.get("sales_channels", [])),
        "expected_current_period_start_date": question["current_period_start_date"],
        "predicted_current_period_start_date": args_predicted.get("current_period_start_date", ""),
        "current_period_start_date_correct": question["current_period_start_date"] == args_predicted.get("current_period_start_date", ""),
        "expected_current_period_end_date": question["current_period_end_date"],
        "predicted_current_period_end_date": args_predicted.get("current_period_end_date", ""),
        "current_period_end_date_correct": question["current_period_end_date"] == args_predicted.get("current_period_end_date", ""),
    }


def eval_questions(llm, tools, questions: list[dict], max_workers=10):
    def task(question: dict):
        llm_resp = llm.generate_with_function_calling(
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": question["question"],
                },
            ],
            tools=tools,
            functions_look_up=functions_look_up,
        )
        llm_resp.update(eval_llm_resp(question, llm_resp))
        return llm_resp

    eval_res = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(task, question) for question in questions]

        for future in tqdm(as_completed(futures), total=len(questions), desc="Evaluating questions"):
            eval_res.append(future.result())

    return eval_res

In [13]:
def calculate_accuracies(df):
    accuracies = {}
    for col in df.columns:
        if col.endswith("_correct"):
            accuracy = df[col].sum() / df.shape[0]
            accuracies[col.replace("correct", "accuracy")] = f"{accuracy:.2%}"
    return accuracies

In [14]:
import pandas as pd

df_openai = pd.DataFrame(eval_questions(llm_openai, tools_openai, questions[:10], max_workers=10))

Evaluating questions: 100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


In [15]:
df_openai

,message,new_messages,model,tool_calls_details,token_usage,execution_time,question,expected_metric,predicted_metric,metric_correct,...,brands_correct,expected_sales_channels,predicted_sales_channels,sales_channels_correct,expected_current_period_start_date,predicted_current_period_start_date,current_period_start_date_correct,expected_current_period_end_date,predicted_current_period_end_date,current_period_end_date_correct
0,{'content': 'The Return on Ad Spend for Luxe L...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_IgRLwuwwKnFVfhZHTO9PMVs0': {'tool_resul...,"{'completion_tokens': 35, 'prompt_tokens': 945...",6.191363,What was the Return on Ad Spend for Luxe Livin...,RETURN_ON_AD_SPEND,RETURN_ON_AD_SPEND,True,...,True,[GOOGLE_SHOPPING],[GOOGLE_SHOPPING],True,2024-02-15,2024-02-15,True,2024-05-15,2024-05-15,True
1,{'content': 'The average time to purchase for ...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_Q4UQ67MRYmJOBlWEi34m57C7': {'tool_resul...,"{'completion_tokens': 26, 'prompt_tokens': 931...",6.195888,Calculate the Average Time to Purchase for Bea...,AVG_TIME_TO_PURCHASE,AVG_TIME_TO_PURCHASE,True,...,True,[OWN_WEBSITE],[],False,2024-05-01,2024-05-01,True,2024-05-31,2024-05-31,True
2,{'content': 'The Repeat Purchase Rate for PetP...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_JXAQikoU3xs2xbdHYTHKdO9l': {'tool_resul...,"{'completion_tokens': 35, 'prompt_tokens': 909...",7.304885,What was the Repeat Purchase Rate for PetPal o...,REPEAT_PURCHASE_RATE,REPEAT_PURCHASE_RATE,True,...,True,[AMAZON],[],False,2024-03-01,2024-03-01,True,2024-05-31,2024-05-31,True
3,{'content': 'The Customer Acquisition Cost for...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_1xsobTEEXKQI3ainTGMtpefj': {'tool_resul...,"{'completion_tokens': 28, 'prompt_tokens': 100...",8.238434,Calculate the Customer Acquisition Cost for Ec...,CUSTOMER_ACQUISITION_COST,CUSTOMER_ACQUISITION_COST,True,...,True,"[AMAZON, ETSY]",[ETSY],False,2024-04-01,2024-04-01,True,2024-04-30,2024-04-30,True
4,{'content': 'The total revenue for Nike and Ad...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_tCh5KEFVg8IvKJB5WNtN8qFC': {'tool_resul...,"{'completion_tokens': 35, 'prompt_tokens': 100...",8.250222,What was the Total Revenue for Nike and Adidas...,TOTAL_REVENUE,TOTAL_REVENUE,True,...,True,"[AMAZON, EBAY]","[AMAZON, EBAY]",True,2024-01-01,2024-01-01,True,2024-03-31,2024-03-31,True
5,{'content': 'The Cart Abandonment Rate for IKE...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_tcn9FaulIwwN0rvpGLQKdNh5': {'tool_resul...,"{'completion_tokens': 39, 'prompt_tokens': 951...",8.245225,Determine the Cart Abandonment Rate for IKEA o...,CART_ABANDONMENT_RATE,CART_ABANDONMENT_RATE,True,...,True,"[OWN_WEBSITE, PHYSICAL_STORES]","[OWN_WEBSITE, PHYSICAL_STORES]",True,2024-05-01,2024-05-01,True,2024-06-09,2024-06-09,True
6,{'content': 'The Net Profit Margin for Gourmet...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_UwvBIFPETRqe9BnMbj9Rv9Zy': {'tool_resul...,"{'completion_tokens': 26, 'prompt_tokens': 930...",8.239079,Determine the Net Profit Margin for GourmetDel...,NET_PROFIT_MARGIN,NET_PROFIT_MARGIN,True,...,True,[OWN_WEBSITE],[OWN_WEBSITE],True,2024-04-01,2024-04-01,True,2024-06-09,2024-06-30,False
7,{'content': 'The Average Order Value for Apple...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_5fhZGHzuJSpe9SLJuyLaRF62': {'tool_resul...,"{'completion_tokens': 35, 'prompt_tokens': 904...",8.249948,Calculate the Average Order Value for Apple pr...,AVG_ORDER_VALUE,AVG_ORDER_VALUE,True,...,True,[OWN_WEBSITE],[OWN_WEBSITE],True,2024-04-01,2024-04-01,True,2024-05-31,2024-05-31,True
8,{'content': 'The Customer Lifetime Value for F...,"[{'content': None, 'role': 'assistant', 'tool_...",gpt-3.5-turbo-0125,{'call_JGC7IRA8MCjx6GPtCMFK3Q55'

In [16]:
calculate_accuracies(df_openai)

{'metric_accuracy': '100.00%',
 'brands_accuracy': '100.00%',
 'sales_channels_accuracy': '70.00%',
 'current_period_start_date_accuracy': '100.00%',
 'current_period_end_date_accuracy': '80.00%'}

In [17]:
df_anthropic = pd.DataFrame(eval_questions(llm_anthropic, tools_anthropic, questions[:10], max_workers=3))

Evaluating questions: 100%|██████████| 10/10 [00:45<00:00,  4.58s/it]


In [18]:
df_anthropic

,message,new_messages,model,tool_calls_details,token_usage,execution_time,question,expected_metric,predicted_metric,metric_correct,...,brands_correct,expected_sales_channels,predicted_sales_channels,sales_channels_correct,expected_current_period_start_date,predicted_current_period_start_date,current_period_start_date_correct,expected_current_period_end_date,predicted_current_period_end_date,current_period_end_date_correct
0,"{'content': 'Based on the data retrieved, the ...","[{'role': 'assistant', 'content': [{'text': ""T...",claude-3-5-sonnet-20240620,{'toolu_01XoZxrjrLvmkhmnTWyejrrj': {'tool_resu...,"{'completion_tokens': 70, 'prompt_tokens': 202...",11.722178,What was the Total Revenue for Nike and Adidas...,TOTAL_REVENUE,TOTAL_REVENUE,True,...,True,"[AMAZON, EBAY]","[AMAZON, EBAY]",True,2024-01-01,2024-01-01,True,2024-03-31,2024-03-31,True
1,"{'content': 'Based on the data retrieved, I ca...","[{'role': 'assistant', 'content': [{'text': ""C...",claude-3-5-sonnet-20240620,{'toolu_01NkNLhdVJRv2qobTQFCJgrL': {'tool_resu...,"{'completion_tokens': 111, 'prompt_tokens': 18...",12.927568,Calculate the Average Order Value for Apple pr...,AVG_ORDER_VALUE,AVG_ORDER_VALUE,True,...,True,[OWN_WEBSITE],[OWN_WEBSITE],True,2024-04-01,2024-04-01,True,2024-05-31,2024-05-31,True
2,"{'content': 'Based on the data retrieved, the ...","[{'role': 'assistant', 'content': [{'text': 'T...",claude-3-5-sonnet-20240620,{'toolu_01RkAsQ1yvp1nd7DP7hRBwGB': {'tool_resu...,"{'completion_tokens': 121, 'prompt_tokens': 18...",13.271748,What was the Conversion Rate for Samsung on Wa...,CONVERSION_RATE,CONVERSION_RATE,True,...,True,[WALMART_MARKETPLACE],[WALMART_MARKETPLACE],True,2024-01-01,2024-01-01,True,2024-03-31,2024-03-31,True
3,"{'content': 'Now, I can provide you with the C...","[{'role': 'assistant', 'content': [{'text': ""C...",claude-3-5-sonnet-20240620,{'toolu_011JNiDbSVDwSw2DBuTvNTZc': {'tool_resu...,"{'completion_tokens': 267, 'prompt_tokens': 18...",13.075303,Determine the Cart Abandonment Rate for IKEA o...,CART_ABANDONMENT_RATE,CART_ABANDONMENT_RATE,True,...,True,"[OWN_WEBSITE, PHYSICAL_STORES]","[OWN_WEBSITE, PHYSICAL_STORES]",True,2024-05-01,2024-05-01,True,2024-06-09,2024-06-09,True
4,"{'content': 'Based on the data retrieved, the ...","[{'role': 'assistant', 'content': [{'text': ""T...",claude-3-5-sonnet-20240620,{'toolu_01WJwTfyG8GSkDoCp9XKjGNH': {'tool_resu...,"{'completion_tokens': 203, 'prompt_tokens': 19...",13.520923,What was the Customer Lifetime Value for FitFl...,CUSTOMER_LIFETIME_VALUE,CUSTOMER_LIFETIME_VALUE,True,...,True,[INSTAGRAM_SHOPPING],[INSTAGRAM_SHOPPING],True,2024-05-11,2024-05-11,True,2024-06-09,2024-06-10,False
5,"{'content': 'Based on the data retrieved, the ...","[{'role': 'assistant', 'content': [{'text': ""C...",claude-3-5-sonnet-20240620,{'toolu_01RD1voxMDKuzbY3uhHEWTJP': {'tool_resu...,"{'completion_tokens': 119, 'prompt_tokens': 19...",13.556440,Calculate the Customer Acquisition Cost for Ec...,CUSTOMER_ACQUISITION_COST,CUSTOMER_ACQUISITION_COST,True,...,True,"[AMAZON, ETSY]","[AMAZON, ETSY]",True,2024-04-01,2024-04-01,True,2024-04-30,2024-04-30,True
6,{'content': 'Based on the data retrieved from ...,"[{'role': 'assistant', 'content': [{'text': ""T...",claude-3-5-sonnet-20240620,{'toolu_014ijpDf2fpVfyqbKurqa2pv': {'tool_resu...,"{'completion_tokens': 129, 'prompt_tokens': 19...",11.126449,What was the Return on Ad Spend for Luxe Livin...,RETURN_ON_AD_SPEND,RETURN_ON_AD_SPEND,True,...,True,[GOOGLE_SHOPPING],[GOOGLE_SHOPPING],True,2024-02-15,2024-02-15,True,2024-05-15,2024-05-15,True
7,{'content': 'The Net Profit Margin for Gourmet...,"[{'role': 'assistant', 'content': [{'text': ""T...",claude-3-5-sonnet-20240620,{'toolu_01Bhico4fLCAqDByirH3hr5k': {'tool_resu...,"{'completion_tokens': 130, 'prompt_tokens': 19...",11.670891,Determine the Net Profit Margin for GourmetDel...,NET_PROFIT_MARGIN,NET_PROFIT_MARGIN,True,...,True,[OWN_WEBSITE],[OWN_WEBSITE],True,2024-04-01,2024-04-01,True,2024-06-09,2024-06-30,False
8,"

In [19]:
calculate_accuracies(df_anthropic)

{'metric_accuracy': '90.00%',
 'brands_accuracy': '90.00%',
 'sales_channels_accuracy': '90.00%',
 'current_period_start_date_accuracy': '90.00%',
 'current_period_end_date_accuracy': '70.00%'}